### 3.1.	Zbiór MovieLens, badania eksploracyjne i edycja zbioru danych

In [11]:
# Import bibliotek
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input,  Dot,Dropout, Dense, BatchNormalization, Concatenate
from tensorflow.keras.utils import model_to_dot
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from IPython.display import SVG
from tensorflow.keras.constraints import NonNeg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import recmetrics
import seaborn as sns
#from sklearn.preprocessing import StandardScaler

In [3]:
# Wczytanie zbioru movielens
ratings_df = pd.read_csv('./ml-latest-small/ratings.csv', header=0, names=['user_id', 'movie_id', 'rating', 'timestamp'])
movies_df = pd.read_csv('./ml-latest-small/movies.csv', header=0, names=['movieId', 'title', 'genres'])
tags_df = pd.read_csv('./ml-latest-small/tags.csv', header=0, names=['user_id', 'movie_id', 'tag', 'timestamp'])

In [6]:
ratings_df.iloc[:5]

,user_id,movie_id,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [9]:
movies_df.iloc[:5]

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [10]:
tags_df.iloc[:5]

,user_id,movie_id,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200


Długi ogon - wykres